In [ ]:
!pip install pandas 
!pip install psycopg2-binary
!pip install sqlalchemy

In [1]:
import json
from kafka import KafkaConsumer
import psycopg2
from psycopg2.extras import execute_values

# PostgreSQL database connection details
db_config = {
    'dbname': '*',
    'user': '*',
    'password': '*',
    'host': '*',
    'port': 5432
}

# Kafka topic
postgres_topic = "postgresql"
brokers = ["*:9092"]

# Create a Kafka consumer
consumer = KafkaConsumer(
    postgres_topic,
    bootstrap_servers=brokers,
    value_deserializer=lambda message: json.loads(message.decode('utf-8'))
)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

try:
    print("Starting to consume messages.")
    for message in consumer:
        data = message.value
        data['datatype'] = 2
        print(f"Received message: {data}")
        # Insert data into PostgreSQL
        try:
            columns = data.keys()
            values = [tuple(data.values())]
            insert_query = f"INSERT INTO malware_data ({', '.join(columns)}) VALUES %s"
            execute_values(cursor, insert_query, values)
            conn.commit()
            print("Data inserted successfully")
        except Exception as e:
            print(f"Failed to insert data: {e}")
            conn.rollback()
except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    # Close PostgreSQL connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("PostgreSQL connection closed.")


Connected to PostgreSQL successfully.
Starting to consume messages.
Received message: {'SizeOfOptionalHeader': '-0.36050749977742014', 'MinorLinkerVersion': '-0.37628545401339736', 'SizeOfCode': '0.7319818837332536', 'SizeOfInitializedData': '-0.21890447616440128', 'SizeOfUninitializedData': '-0.12754141233745858', 'BaseOfCode': '-0.10815786799743762', 'BaseOfData': '0.8442979845835227', 'ImageBase': '-0.014265954069590235', 'SectionAlignment': '-0.02785883454533931', 'MajorOperatingSystemVersion': '-3.783301043371495', 'MinorImageVersion': '-0.08601248526438687', 'MajorSubsystemVersion': '-8.146929400017378', 'SizeOfHeaders': '-0.19993743506016834', 'CheckSum': '-0.30014776191072096', 'Subsystem': '18.955819550476978', 'DllCharacteristics': '-1.18386423766959', 'SizeOfStackReserve': '0.36320900044581794', 'SizeOfStackCommit': '-0.18911824926091036', 'SizeOfHeapReserve': '0.1275489610136435', 'NumberOfRvaAndSizes': '-0.006541198444001456', 'SectionsNb': '1.685708547402764', 'SectionsMi